In [2]:
# import matplotlib.pyplot as plt
import plotly.graph_objects as go
from utils import data

In [3]:
%run exploration.ipynb

id:
	 Unique count: 10841
	 Too Many Values

App:
	 Unique count: 9660
	 Too Many Values

Category:
	 Unique count: 34
	 ['ART_AND_DESIGN' 'AUTO_AND_VEHICLES' 'BEAUTY' 'BOOKS_AND_REFERENCE'
 'BUSINESS' 'COMICS' 'COMMUNICATION' 'DATING' 'EDUCATION' 'ENTERTAINMENT'
 'EVENTS' 'FINANCE' 'FOOD_AND_DRINK' 'HEALTH_AND_FITNESS' 'HOUSE_AND_HOME'
 'LIBRARIES_AND_DEMO' 'LIFESTYLE' 'GAME' 'FAMILY' 'MEDICAL' 'SOCIAL'
 'SHOPPING' 'PHOTOGRAPHY' 'SPORTS' 'TRAVEL_AND_LOCAL' 'TOOLS'
 'PERSONALIZATION' 'PRODUCTIVITY' 'PARENTING' 'WEATHER' 'VIDEO_PLAYERS'
 'NEWS_AND_MAGAZINES' 'MAPS_AND_NAVIGATION' '1.9']

Rating:
	 Unique count: 41
	 [ 4.1  3.9  4.7  4.5  4.3  4.4  3.8  4.2  4.6  3.2  4.   nan  4.8  4.9
  3.6  3.7  3.3  3.4  3.5  3.1  5.   2.6  3.   1.9  2.5  2.8  2.7  1.
  2.9  2.3  2.2  1.7  2.   1.8  2.4  1.6  2.1  1.4  1.5  1.2 19. ]

Reviews:
	 Unique count: 6002
	 Too Many Values

Size:
	 Unique count: 462
	 Too Many Values

Installs:
	 Unique count: 22
	 ['10,000+' '500,000+' '5,000,000+' '50,000,

In [4]:
%run cleaning.ipynb

C:\Users\clement\AppData\Local\Temp\ipykernel_9416\3656646732.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Last Updated"] = data["Last Updated"].apply(lambda x: parse(x))
C:\Users\clement\AppData\Local\Temp\ipykernel_9416\2713534833.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Installs'] = data['Installs'].str.replace('[+,]', '', regex=True).astype(int)
C:\Users\clement\AppData\Local\Temp\ipykernel_9416\90886282.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy o

## Moyenne du nombre de téléchargements par catégorie d'applications

Préparation des données

In [5]:
# Calculer le nombre total de téléchargements par catégorie
total_downloads_by_category = data.groupby('Category')['Installs'].sum().sort_values()
# Calculer le nombre total d'applications par catégorie
total_apps_by_category = data['Category'].value_counts()
# Calculer la moyenne des téléchargements par catégorie
average_downloads_by_category = total_downloads_by_category / total_apps_by_category
# Trier les catégories d'applications par ordre croissant de moyenne de téléchargements
average_downloads_by_category = average_downloads_by_category.sort_values(ascending=True)

Création du graphe

In [6]:
fig = go.Figure(data=[go.Bar(
    x=average_downloads_by_category.index,
    y=average_downloads_by_category
)])
fig.update_layout(
    title_text='Moyenne du nombre de téléchargements par catégorie d\'applications',
    xaxis_title="Catégorie", 
    yaxis_title="Moyenne du nombre de téléchargements", 
    width=900,
    xaxis=dict(tickmode = 'linear', dtick = 1),
    title_x=0.5
)
fig.show()

# Nombre d'application gratuite et payante

Préparation des données

In [7]:
types = data["Type"]
type_counts = types.value_counts()

Création du graphe

In [8]:
fig = go.Figure(data=[go.Pie(
    labels=type_counts.index,
    values=type_counts
)])
fig.update_layout(
    title_text='Nombre d\'application gratuite et payante',
    width=500,
    title_x=0.5
)
fig.show()

# Nombre d'applications par catégorie

Obtention du nombre d'applications par catégorie

In [9]:
category = data['Category'].value_counts()

Affichage du nombre d'applications par catégorie dans un graphique à barres horizontales

In [10]:
fig = go.Figure(data=[go.Bar(
    y=category.index,
    x=category,
    orientation='h'
)])
fig.update_layout(
    title_text='Nombre d\'applications par catégorie',
    xaxis_title="Nombre d'applications", 
    width=900,
    title_x=0.5
)
fig.show()

# Meilleures catégories par note sur le contenu

Préparation des données

In [11]:
# Groupe les valeurs par note de sur le contenu
grouped_categories = data.groupby('Content Rating')['App'].count()
top_categories = grouped_categories.sort_values(ascending=False)

Création du graphe

In [12]:
fig = go.Figure(data=[go.Pie(
    labels=top_categories.index,
    values=top_categories
)])
fig.update_layout(
    title_text='Meilleures catégories par note sur le contenu',
    width=500,
    title_x=0.5
)
fig.show()

# Nombre d\'application par note

In [13]:
ratings_count = data['Rating'].value_counts()

In [14]:
fig = go.Figure(data=[go.Bar(
    x=ratings_count.index,
    y=ratings_count
)])
fig.update_layout(
    title_text='Nombre d\'application par note',
    xaxis_title="Note", 
    yaxis_title="Nombre d'application à cette note", 
    width=900,
    xaxis=dict(tickmode = 'linear', dtick = 1),
    title_x=0.5
)
fig.show() 

# Pourcentage d'avis posté comparé au nombre d'installations

In [15]:
# Groupe les valeurs par note de sur le contenu
reviews_and_unreviewed = data[['Reviews', 'Unreviewed']].sum()

In [16]:
fig = go.Figure(data=[go.Pie(
    labels=reviews_and_unreviewed.index,
    values=reviews_and_unreviewed,
    
)])
fig.update_layout(
    title_text='Pourcentage d\'avis posté comparé au<br>nombre d\'installations',
    width=500,
    title_x=0.5
)
fig.show()